In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
import netgen.gui

import matplotlib.pyplot as plt
from time import time

# Example 11.1

Simulate the Navier-Stokes equations on a channel with a step-like cross-section extension:
$\Omega = (0; 2]\times(0; 1) \cup (2; 10)\times(−1; 1)$. 

- On the left side we have a parabolic inflow profile,

- on the right side natural boundary conditions (outlet), thus $(-\nu\ \nabla u + pI)\cdot n = 0$.

- Everywhereelse we have wall boundary conditions (homogeneous Dirichlet bc). 

Choose a viscosity $\nu = 10^{-3}$ and a zero right hand side and use the Taylor-Hood element and the IMEX scheme as in the tutorial example navierstokes.py. 

Increase the inflow velocity and present an interpretation of your results. For this you might google "backward facing step, navier stokes". Further note, that you might need to adapt the time step size τ.

In [2]:
#Ω = (0, 2]×(0, 1) u (2, 10)×(−1, 1)
                  
geo = SplineGeometry()
# Left rectangle
a1 = geo.AppendPoint(0,0)
b1 = geo.AppendPoint(2,0)
c1 = geo.AppendPoint(2,1)
d1 = geo.AppendPoint(0,1)
# Right rectangle
a2 = geo.AppendPoint(2,-1)
b2 = geo.AppendPoint(10,-1)
c2 = geo.AppendPoint(10,1)
d2 = geo.AppendPoint(2,1)

p1_seq = [a1, b1, a2, b2, c2, c1, d1]
p2_seq = [b1, a2, b2, c2, c1, d1, a1]
bc_seq = ["wall"]*3 + ["outlet"] + ["wall"]*2 + ["inlet"]

for p1, p2, bc in zip(p1_seq, p2_seq, bc_seq):
    geo.Append(["line", p1, p2], leftdomain=1, rightdomain=0, bc=bc)

#geo.Append(["line", a1, b1], leftdomain=1, rightdomain=0, bc="wall")#
#geo.Append(["line", b1, a2], leftdomain=1, rightdomain=0, bc="wall")



#                                       bottom, right side, top, left side
#geo.AddRectangle((0, 0), (2, 1), bcs = ("wall", "outlet", "wall", "inlet"))
#geo.AddRectangle((2, -1), (10, 1), bcs = ("wall", "outlet", "wall", "inlet"))
#geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl", maxh=0.02)
mesh = Mesh(geo.GenerateMesh(maxh=0.3))


Draw(mesh)

In [3]:
mesh.Curve(3)
Draw(mesh)

In [4]:
# Parameters
nu = Parameter(1e-3)
rhs = Parameter(0.0)


V = VectorH1(mesh,order=3, dirichlet="wall|inlet")
Q = H1(mesh,order=2)

X = FESpace([V,Q])

u,p = X.TrialFunction()
v,q = X.TestFunction()

stokes = nu * InnerProduct(grad(u), grad(v)) + (div(u)*q + div(v)*p) - 1e-10*p*q
a = BilinearForm(X)
a += stokes *dx
a.Assemble()

# nothing here ...
f = LinearForm(X)
#f += rhs*v*dx
f.Assemble()

In [7]:
# gridfunction for the solution
gfu = GridFunction(X)

# parabolic inflow at inlet:
width = 1
v_in = 0.5
tau = 0.001
uin = CoefficientFunction( (v_in*y*(1-y), 0) ) # uin...initial u
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))


# solve Stokes problem for initial conditions:
inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

# for visualization
print("Showing initial solution")
Draw (Norm(gfu.components[0]), mesh, "velocity", sd=3)
Draw (Norm(gfu.components[1]), mesh, "pressure")

Showing initial solution


In [ ]:
# timestepping parameters
#tau = 0.00005 #Parameter(0.0005)
tend = 20
#tau.Set()
# matrix for implicit Euler 
mstar = BilinearForm(X)
mstar += SymbolicBFI(u*v + tau*stokes)
mstar.Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky")

# the non-linear term 
conv = BilinearForm(X, nonassemble = True)
conv += (grad(u) * u) * v * dx

# implicit Euler/explicit Euler splitting method:
t = 0
rt = time()
with TaskManager():
    while t < tend:
        print ("t=", t, end="\r")

        conv.Apply(gfu.vec, res)
        res.data += a.mat*gfu.vec
        gfu.vec.data -= tau * inv * res    

        t = t + tau
        Redraw()
rt = time() - rt
print()
rt

t= 11.79399999999890376 0.05100000000000004 0.07900000000000006 0.11900000000000009 0.1350000000000001 0.16300000000000012 0.2930000000000002 0.32900000000000024 0.36000000000000026 0.3900000000000003 0.4130000000000003 0.45400000000000035 0.5550000000000004 0.5890000000000004 0.6310000000000004 0.6710000000000005 0.7240000000000005 0.7540000000000006 0.8470000000000006 0.8650000000000007 0.8920000000000007 0.9520000000000007 0.9800000000000008 1.0030000000000003 1.0619999999999938 1.0889999999999909 1.113999999999988 1.1749999999999814 1.1989999999999787 1.2269999999999757 1.29199999999996851.351999999999962 1.3819999999999586 1.4759999999999482 1.4949999999999461 1.5199999999999434 1.587999999999936 1.632999999999931 1.6559999999999284 1.6789999999999259 1.696999999999924 1.7209999999999213 1.7789999999999149 1.8019999999999123 1.8999999999999015 1.9359999999998976 1.9629999999998946 1.9819999999998925 2.0109999999998895 2.0479999999998855 2.070999999999883 2.1579999999998734 2.18299

Do mochts an Wüabö!